In [1]:
#Import required library
import pandas as pd
import numpy as np

import elasticsearch
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm


In [2]:
!elasticsearch -version

'elasticsearch' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
#Load products and category
df_products = pd.read_json("products_scraper.json")
df_products

,product_description,category
0,Antenna booster for Mercedes-Benz E-Class 210 ...,amplifiers
1,Amplifier for Audi A6 Sedan C6 (05.2004 - 03.2...,amplifiers
2,Amplifier for Audi A6 Avant C6 (03.2005 - 08.2...,amplifiers
3,Antenna booster for Volkswagen Passat V Varian...,amplifiers
4,Amplifier for Volkswagen Phaeton Sedan (04.200...,amplifiers
...,...,...
5971,Cassette player for Seat Ibiza I Hatchback (06...,cassette_and_cd_players
5972,CD player for Renault Laguna II Grandtour (03....,cassette_and_cd_players
5973,CD player and climate control panel for Mazda ...,cassette_and_cd_players
5974,CD player for Renault Laguna II Grandtour (03....,cassette_and_cd_players


In [5]:
#Take few records for experiment
#df_products = df_products.sample(frac=1).reset_index(drop=True)
df_products = df_products[1:20]

In [6]:
df_products

,product_description,category
1,Amplifier for Audi A6 Sedan C6 (05.2004 - 03.2...,amplifiers
2,Amplifier for Audi A6 Avant C6 (03.2005 - 08.2...,amplifiers
3,Antenna booster for Volkswagen Passat V Varian...,amplifiers
4,Amplifier for Volkswagen Phaeton Sedan (04.200...,amplifiers
5,"Antenna booster for Rover 75 1.8, 120 hp, seda...",amplifiers
6,"Amplifier for Honda Accord VI 1.8, 136 hp, sed...",amplifiers
7,Amplifier for Volvo XC90 I SUV (06.2002 - 01.2...,amplifiers
8,Amplifier for Audi A4 Sedan B7 (11.2004 - 06.2...,amplifiers
9,Antenna booster for Mercedes-Benz S-Class W220...,amplifiers
10,Antenna booster for Audi A6 Sedan C6 (05.2004 ...,amplifiers


In [7]:
import string


# nltk library Stop words list
stop_words = {'hasnt', 'for', 'ma', 'up', 'should', 'which', 'now', 'her', 'so', 'these', 'don', 'll', 'youd', 'against', 'doing', 'my', 'mightnt', 'him', 'but', 'is', 'dont', 'shouldve', 'arent', 'then', 'during', 't', 'above', 'once', 'shouldn', 'we', 'themselves', 're', 'was', 'needn', 'herself', 'has', 'be', 'as', 'from', 'until', 'between', 'his', 'hadn', 'mustn', 'under', 'too', 'through', 'mustnt', 'can', 'ours', 'theirs', 'me', 'you', 'shouldnt', 'she', 'over', 'or', 'isn', 'in', 'your', 'haven', 'ourselves', 'again', 'further', 'when', 'no', 'o', 'he', 'what', 'himself', 'all', 'after', 'will', 'been', 'have', 'not', 'being', 'other', 'having', 'few', 'both', 'than', 'that', 'it', 'some', 'about', 'their', 'whom', 'its', 'are', 'had', 'out', 'into', 'where', 've', 'our', 'the', 'youve', 'them', 'nor', 'just', 'while', 'am', 'down', 'd', 'm', 'of', 'doesnt', 'why', 'hers', 'shant', 'wasn', 'havent', 'hadnt', 'aren', 'wouldnt', 'who', 'by', 'here', 'shan', 'didn', 'such', 'own', 'below', 'neednt', 'same', 'if', 'off', 'myself', 'a', 'each', 'this', 'thatll', 'how', 'youre', 'does', 'yourselves', 'do', 'very', 'isnt', 'any', 'wont', 'werent', 'those', 'because', 'yourself', 'y', 'won', 'did', 'at', 'couldnt', 'more', 'its', 'there', 'with', 'on', 'itself', 'only', 'an', 'before', 'mightn', 'yours', 'ain', 'they', 'wasnt', 'were', 'doesn', 'shes', 'weren', 'most', 'didnt', 'hasn', 'youll', 'wouldn', 'to', 's', 'i', 'couldn', 'and'}


# make all text lowercase
def text_lowercase(text):
    return text.lower()


# remove stopwords
def remove_stopwords(text):
    words = text.split()
    text = [i for i in words if not i in stop_words]
    text = ' '.join(text)
    return text


# remove punctuation
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)


# preprocess text string
def preprocessing(text):
    text = text_lowercase(text)
    text = remove_punctuation(text)
    text = remove_stopwords(text)
    return text


In [9]:
model = SentenceTransformer('all-MiniLM-L6-v2')
def get_vector(product_description,category=""):
    productandcategory = product_description+category
    product_details = preprocessing(productandcategory)
    products  = [productandcategory]
    products_embeddings = model.encode(products)
    _ = list(products_embeddings.flatten())
    encod_np_array = np.array(_)
    products_embeddings_list = encod_np_array.tolist()
    print(len(products_embeddings_list))
    return products_embeddings_list

In [10]:

tqdm.pandas()
df_products["product_vector"] = df_products.progress_apply(lambda x: get_vector(x.product_description, x.category), axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 106.67it/s]

384
384
384
384
384
384
384
384
384
384
384
384
384
384
384
384
384
384
384



C:\Users\I331215\AppData\Local\Temp\ipykernel_30232\1055724507.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_products["product_vector"] = df_products.progress_apply(lambda x: get_vector(x.product_description, x.category), axis=1)


In [ ]:
#df_products=df_products.rename(columns={'products_vector':'product_vector'})

In [11]:
products_data = df_products.to_dict("records")



In [12]:
len(products_data)

19

In [13]:
#Establish connection with ElasticSearch
#client = Elasticsearch(['https://my-elastic-search-d01f60.es.us-central1.gcp.cloud.es.io:9243'], http_auth=("elastic", "2M7cicGL0CuM5PXK0gMvAHSo"))

In [14]:
#Establish connection with ElasticSearch
client = Elasticsearch(['http://localhost:9200'], http_auth=("elastic", "NzAQmqmHd7-ouB+fo9-4"))


C:\Users\I331215\AppData\Local\Temp\ipykernel_30232\2225361553.py:2: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  client = Elasticsearch(['http://localhost:9200'], http_auth=("elastic", "NzAQmqmHd7-ouB+fo9-4"))


In [ ]:
client.ping()

In [31]:
client.indices.get_mapping(index="myproducts")

ObjectApiResponse({'myproducts': {'mappings': {'properties': {'category': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'product_description': {'type': 'text'}, 'product_vector': {'type': 'dense_vector', 'dims': 384, 'index': True, 'similarity': 'cosine'}}}}})

In [32]:
#index values to elastic search
for product in products_data:
    client.index(index="myproducts",body=product)

C:\Users\I331215\AppData\Local\Temp\ipykernel_5204\1599758539.py:3: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  client.index(index="myproducts",body=product)


In [ ]:
query = {
    "field": "title_vector",
    "query_vector": [-0.01807806, 0.024579186,...],
    "k": 10,
    "num_candidates": 100
}
es = Elasticsearch(request_timeout=600, hosts='http://localhost:9200')
res = es.knn_search(index="index_name", knn=query, source=["filed1", "field2"])

In [33]:
search_string = input("Enter the Input Query ")
token_vector = get_vector(search_string,search_string)
token_vector

Enter the Input Query antena
384


[-0.004093733616173267,
 0.02357134409248829,
 -0.06676733493804932,
 0.004734553396701813,
 -0.04035312309861183,
 -0.04426214471459389,
 0.052442289888858795,
 0.04255903512239456,
 -0.027889035642147064,
 0.024130670353770256,
 0.028040101751685143,
 -0.01495275180786848,
 -0.10206850618124008,
 -0.004777778871357441,
 -0.03451210632920265,
 0.023740168660879135,
 0.005177771206945181,
 -0.011500303633511066,
 0.06042799726128578,
 0.051443230360746384,
 -0.008246995508670807,
 -0.009661983698606491,
 -0.02543332614004612,
 0.004635734483599663,
 -0.015372026711702347,
 0.12404852360486984,
 -0.022231431677937508,
 0.025433745235204697,
 -0.0076817418448626995,
 -0.07162125408649445,
 -0.05134682357311249,
 0.07172346115112305,
 0.07972430437803268,
 -0.012163601815700531,
 -0.05468189716339111,
 -0.06712175160646439,
 -0.06828362494707108,
 0.011421361938118935,
 0.030866630375385284,
 0.05984216183423996,
 -0.020189717411994934,
 0.023323120549321175,
 -0.0530722476541996,
 -0.014

In [45]:
query = {
    "field": "product_vector",
    "query_vector": token_vector,
    "k": 10,
    "num_candidates": 100
}
data  = client.search(index="myproducts", knn=query, source=["product_description"])['hits']['hits']

In [48]:
data

[{'_index': 'myproducts',
  '_id': 'Efqd9oQBCTIUcWkQvwga',
  '_score': 0.6000775,
  '_source': {'product_description': 'Antenna booster for Hyundai Santa Fe 2.4 16V 4x4, 146 hp, 2003 № 96270-26500 '}},
 {'_index': 'myproducts',
  '_id': 'JPqi9oQBCTIUcWkQkQjd',
  '_score': 0.6000775,
  '_source': {'product_description': 'Antenna booster for Hyundai Santa Fe 2.4 16V 4x4, 146 hp, 2003 № 96270-26500 '}},
 {'_index': 'myproducts',
  '_id': 'Dfqd9oQBCTIUcWkQvgjO',
  '_score': 0.5868694,
  '_source': {'product_description': 'Antenna booster for Volkswagen Phaeton Sedan (04.2002 - 03.2016), № 921837001 '}},
 {'_index': 'myproducts',
  '_id': 'IPqi9oQBCTIUcWkQkQig',
  '_score': 0.5868694,
  '_source': {'product_description': 'Antenna booster for Volkswagen Phaeton Sedan (04.2002 - 03.2016), № 921837001 '}},
 {'_index': 'myproducts',
  '_id': 'Dvqd9oQBCTIUcWkQvgje',
  '_score': 0.58176637,
  '_source': {'product_description': 'Amplifier for Ford Galaxy Minivan I (03.1995 - 05.2006), № 94GP-18B84

In [50]:

[d['_source'] for d in data]

[{'product_description': 'Antenna booster for Hyundai Santa Fe 2.4 16V 4x4, 146 hp, 2003 № 96270-26500 '},
 {'product_description': 'Antenna booster for Hyundai Santa Fe 2.4 16V 4x4, 146 hp, 2003 № 96270-26500 '},
 {'product_description': 'Antenna booster for Volkswagen Phaeton Sedan (04.2002 - 03.2016), № 921837001 '},
 {'product_description': 'Antenna booster for Volkswagen Phaeton Sedan (04.2002 - 03.2016), № 921837001 '},
 {'product_description': 'Amplifier for Ford Galaxy Minivan I (03.1995 - 05.2006), № 94GP-18B849-A '},
 {'product_description': 'Amplifier for Ford Galaxy Minivan I (03.1995 - 05.2006), № 94GP-18B849-A '},
 {'product_description': 'Amplifier for Volkswagen Phaeton Sedan (04.2002 - 03.2016), № 3D0 035 465A '},
 {'product_description': 'Amplifier for Volkswagen Phaeton Sedan (04.2002 - 03.2016), № 3D0 035 465A '},
 {'product_description': 'Antenna booster for Land Rover Range Rover III SUV (03.2002 - 08.2012), № AD-6865 '},
 {'product_description': 'Antenna booster 

In [158]:
search_string = input("Enter the Input Query ")
token_vector = get_vector(search_string,search_string)
token_vector

Enter the Input Query Music
384


[-0.01748097501695156,
 -0.005768690258264542,
 0.00878185871988535,
 -0.016776960343122482,
 -0.08211445063352585,
 0.07177060097455978,
 0.08093174546957016,
 -0.003116709878668189,
 0.018745174631476402,
 -0.02668456919491291,
 -0.021404972299933434,
 -0.0017681518802419305,
 0.0059697506949305534,
 -0.10264510661363602,
 -0.04111940413713455,
 0.07433567941188812,
 0.0018555163405835629,
 0.05818760767579079,
 0.010185342282056808,
 -0.06896243244409561,
 -0.14531685411930084,
 0.02745821513235569,
 -0.07988112419843674,
 0.04989565163850784,
 0.026030341163277626,
 0.09712465107440948,
 -0.04299742355942726,
 0.12089330703020096,
 0.0613662526011467,
 -0.09022867679595947,
 0.016378918662667274,
 0.0806499794125557,
 0.11017932742834091,
 -0.0461612269282341,
 -0.07640494406223297,
 -0.031753987073898315,
 -0.026667963713407516,
 -0.03995757922530174,
 -0.03344300761818886,
 0.00808669626712799,
 -0.012095850892364979,
 0.03314497694373131,
 -0.022959677502512932,
 -0.032581485807

In [156]:
#booster 

In [157]:
# open file in write mode
flag=False
with open(r'vector.txt', 'w') as fp:
    data="["
    flag=False
    for items in token_vector:
        #file.writelines('{}'.format(items))
        if flag:
            data=data+", "
        data = data + str(items)
        flag=True
    data=data + "]"
    fp.write("%s" % data)   
    fp.close()


In [141]:
df_products[df_products['product_description'].str.contains('player ')]

,product_description,category,product_vector
5834,DVD player for Mazda 6 Station Wagon I (08.200...,dvd_tv_receivers,"[-0.022519683465361595, -0.03331909328699112, ..."
5837,DVD player for Renault Espace IV Minivan (11.2...,dvd_tv_receivers,"[0.016947712749242783, -0.022322984412312508, ..."
5839,DVD player for Volkswagen Phaeton Sedan (04.20...,dvd_tv_receivers,"[-0.023305624723434448, 0.006023638881742954, ..."
5841,DVD player for Audi A3 Hatchback I (09.1996 - ...,dvd_tv_receivers,"[-0.024345392361283302, -0.03275288641452789, ..."
5847,DVD player for Nissan Murano I (Z50) (08.2003 ...,dvd_tv_receivers,"[-0.013141465373337269, -0.006211839150637388,..."
5852,DVD player for Mazda 6 (2002-2008) № CN-VM4270A,dvd_tv_receivers,"[0.013845372013747692, 0.014111255295574665, -..."
5853,DVD player for Audi A4 Avant B6 (04.2001 - 12....,dvd_tv_receivers,"[0.03709869086742401, -0.061652541160583496, -..."
5854,DVD player for Land Rover Range Rover III (200...,dvd_tv_receivers,"[0.01374858245253563, -0.09525226801633835, -0..."
5856,DVD player for Mazda 6 Station Wagon I (08.200...,dvd_tv_receivers,"[-0.012917719781398773, 0.002104921033605933, ..."
5860,DVD player for BMW 7 (F02) (2008-2015) № 9 196...,dvd_tv_receivers,"[-0.015397848561406136, 0.010201898403465748, ..."
